In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES = 3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import get_module_info

pd.set_option("display.precision", 2)

In [3]:
device = 'cuda:0'
device

'cuda:0'

In [4]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [5]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device, 
                                                               track_backward=True, memory_unit='KB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.47,0.06,0.57,0.05,32994.5,128.0,545259520.0,33280.0,1500.17
conv_64_512_3x3,0.85,0.04,1.32,0.03,35972.0,1152.0,4690151424.0,295424.0,789.55
bottleneck_64_512_3x3,0.9,0.04,1.64,0.5,36420.0,2624.0,2383218688.0,150048.0,910.29
conv_64_512_3x3_g2,1.96,0.04,2.77,0.03,34032.5,3136.0,11956733952.0,803328.0,1654.19
conv_64_512_3x3_g8,0.7,0.03,1.6,0.48,32210.0,784.0,2994898944.0,201216.0,858.7
conv_padding,0.78,0.06,1.06,0.05,35972.0,89216.0,4840226816.0,295424.0,759.71
conv_nn_Padding,0.86,0.03,1.39,0.03,40324.0,1152.0,4840226816.0,295424.0,813.98


In [6]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.45,0.04,0.3,0.03,36.47,34.0,545259520.0,33280.0,358.33
conv_64_512_3x3,1.0,0.03,1.17,0.03,37.22,34.0,4690151424.0,295424.0,823.43
bottleneck_64_512_3x3,0.87,0.13,2.27,0.1,39.65,34.0,2383218688.0,150048.0,1193.94
conv_64_512_3x3_g2,1.9,0.04,13.47,0.07,37.42,39.2,11956733952.0,803328.0,5365.61
conv_64_512_3x3_g8,1.08,0.03,12.07,0.06,35.64,34.77,2994898944.0,201216.0,4572.19
conv_padding,0.99,0.03,2.1,0.05,38.19,34.0,4840226816.0,295424.0,1133.26
conv_nn_Padding,1.07,0.05,1.47,0.04,42.25,34.0,4840226816.0,295424.0,974.17


In [7]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device,
                                                               track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.46,0.05,0.63,0.15,0.04,0.03,545259520.0,33280.0,498.2
conv_64_512_3x3,0.57,0.05,0.74,0.1,0.04,0.03,4690151424.0,295424.0,565.32
bottleneck_64_512_3x3,0.82,0.04,1.41,0.16,0.04,0.03,2383218688.0,150048.0,919.77
conv_64_512_3x3_g2,10.55,0.05,12.03,0.04,0.04,0.04,11956733952.0,803328.0,7786.82
conv_64_512_3x3_g8,2.57,0.03,10.06,0.05,0.03,0.03,2994898944.0,201216.0,4384.67
conv_padding,0.53,0.03,0.67,0.11,0.04,0.03,4840226816.0,295424.0,518.28
conv_nn_Padding,0.61,0.05,0.72,0.1,0.04,0.03,4840226816.0,295424.0,591.66


In [8]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.47,0.05,0.66,0.16,37980672.0,35654144.0,545259520.0,33280.0,501.46
conv_64_512_3x3,0.58,0.05,0.76,0.11,38805504.0,35654144.0,4690151424.0,295424.0,573.22
bottleneck_64_512_3x3,0.81,0.04,1.41,0.09,39493120.0,35654144.0,2383218688.0,150048.0,911.85
conv_64_512_3x3_g2,10.51,0.06,11.88,0.13,38068736.0,41105920.0,11956733952.0,803328.0,7727.38
conv_64_512_3x3_g8,2.56,0.03,9.97,0.1,36202496.0,36454400.0,2994898944.0,201216.0,4342.38
conv_padding,0.57,0.05,0.7,0.1,39845888.0,35654144.0,4840226816.0,295424.0,539.93
conv_nn_Padding,0.64,0.06,0.7,0.09,44304384.0,35654144.0,4840226816.0,295424.0,595.37
